In [3]:
import pandas as pd
import numpy as np
import pyod
np.set_printoptions(precision=3)

In [5]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from pyod.models.knn import KNN

In [7]:
testing_set = pd.read_csv('testing_set.csv',sep=',',header=0)

/home/teste/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [26]:
testingSet = testing_set.values
for i in range (99839):
    if(testingSet[i, 84] == "ddos"):
        testingSet[i, 84] = 1
    else:
        testingSet[i, 84] = 0

#np.random.shuffle(testingSet)


#testingSet = np.delete(testingSet, 16, axis=1)
#testingSet = np.delete(testingSet, 17, axis=1)

testingSet = np.delete(testingSet, 7, axis=1)
testingSet = np.delete(testingSet, 4, axis=1)
testingSet = np.delete(testingSet, 2, axis=1)
testingSet = np.delete(testingSet, 1, axis=1)
testingSet = np.delete(testingSet, 0, axis=1)

testingSet = testingSet.astype(float)

for i in range (99838, 0, -1):
    for j in range(0, 77):
        testingSet[i, j] = float(testingSet[i, j])
        if (np.isinf(testingSet[i, j]) or np.isnan(testingSet[i, j])):
            #print("Dropping (",i,",",j,")",testingSet[i, j])
            testingSet = np.delete(testingSet, i, axis=0)
            #testingSet[i, j] = 1.7*np.e**308
            
y_test = testingSet[:, 79].astype(int)
testingSet = np.delete(testingSet, 79, axis=1)

In [28]:
print(testingSet.shape)
print(np.sum(y_test)/testingSet.shape[0])

(99227, 79)
0.17293680147540488


In [29]:
clf_name = 'KNN'
clf = KNN(contamination = 0.17293680147540488)
clf.fit(testingSet) 

KNN(algorithm='auto', contamination=0.17293680147540488, leaf_size=30,
  method='largest', metric='minkowski', metric_params=None, n_jobs=1,
  n_neighbors=5, p=2, radius=1.0)

In [30]:
y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
y_train_scores = clf.decision_scores_  # raw outlier scores

y_test_pred = clf.predict(testingSet)  # outlier labels (0 or 1)
y_test_scores = clf.decision_function(testingSet)  # outlier scores

In [31]:
from pyod.utils.data import evaluate_print

print("\nOn Test Data:")
truePositive = 0
trueNegative = 0
falsePositive = 0
falseNegative = 0

for i in range(y_test.size):
    if(y_test[i] == 1 and y_test_pred[i] == 1):
        truePositive = truePositive+1
    elif (y_test[i] == 0 and y_test_pred[i] == 0):
        trueNegative = trueNegative+1
    elif (y_test[i] == 0 and y_test_pred[i] == 1):
        falsePositive = falsePositive+1
    else:
        falseNegative = falseNegative+1

print(truePositive, " | ", falsePositive)
print(falseNegative, " | ", trueNegative,"\n")

evaluate_print(clf_name, y_test, y_test_scores)


On Test Data:
1363  |  15117
15797  |  66950 

KNN ROC:0.4599, precision @ rank n:0.0824
